# Fast Agent - Build a SQL Agent fast!
# 빠른 에이전트 - SQL 에이전트를 빠르게 만들어보세요!

<img src="./assets/LC_L1_top.png" align="left" width="500">

## Setup
## 설정

Load and/or check for needed environmental variables

필요한 환경 변수를 로드하거나 확인합니다

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env, doublecheck_pkgs

# Load environment variables from .env 
load_dotenv()

# Check and print results
doublecheck_env("example.env")  # check environmental variables
doublecheck_pkgs(pyproject_path="pyproject.toml", verbose=True)   # check packages

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

Define the runtime context to provide the agent and tools with access to the database.

에이전트와 도구가 데이터베이스에 접근할 수 있도록 런타임 컨텍스트를 정의합니다.

In [ ]:
from dataclasses import dataclass

from langchain_community.utilities import SQLDatabase


# define context structure to support dependency injection
@dataclass
class RuntimeContext:
    db: SQLDatabase

<b>⚠️ Security Note:</b> This demo does not include a filter on LLM-generated commands. In production, you would want to limit the scope of LLM-generated commands. ⚠️

<b>⚠️ 보안 참고:</b> 이 데모에는 LLM이 생성한 명령에 대한 필터가 포함되어 있지 않습니다. 프로덕션 환경에서는 LLM이 생성한 명령의 범위를 제한해야 합니다. ⚠️

This tool will connect to the database. Note the use of `get_runtime` to access the graph **runtime context**.

이 도구는 데이터베이스에 연결됩니다. 그래프의 **런타임 컨텍스트**에 접근하기 위해 `get_runtime`을 사용하는 것에 주목하세요.

In [ ]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

Add a system prompt to define your agents behavior.

에이전트의 동작을 정의하는 시스템 프롬프트를 추가합니다.

tool이 오류를 반환하면 SQL을 수정한 다음 다시 시도하라는 부분이 유용함.

Read-Only라고 적어놔도 방심하면 안됨. 그래서 나중에 나오는 HITL 같은걸 사용하거나 전처리기를 사용함. 모든걸 AI에 맡기면 좋지않음.

In [ ]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

Create your agent! Add a model, tools, a prompt, and the runtime access, and go!  You can choose many agents from our [integrations](https://docs.langchain.com/oss/python/integrations/providers) list.

에이전트를 생성하세요! 모델, 도구, 프롬프트, 런타임 접근을 추가하면 됩니다! [통합 목록](https://docs.langchain.com/oss/python/integrations/providers)에서 다양한 에이전트를 선택할 수 있습니다.

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

Here's a display of the agent ReAct Loop.

에이전트의 ReAct 루프를 시각적으로 보여줍니다.

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

Run some queries. Notice:
- The agent does not have the database schema and will need to discover it independently.
- The agent may make mistakes! By returning error messages, the agent can self-correct its queries.
- Notice you invoke the agent with `agent.stream`.
    - This command and the `pretty_print` display the **messages** that communicate information between the model and the tools.
- Notice the agent doesn't remember the schema between invocations... More on this later!

몇 가지 쿼리를 실행해보세요. 다음 사항을 주목하세요:
- 에이전트는 데이터베이스 스키마를 가지고 있지 않으며, 스스로 이를 발견해야 합니다.
- 에이전트는 실수를 할 수 있습니다! 오류 메시지를 반환함으로써 에이전트는 쿼리를 스스로 수정할 수 있습니다.
- `agent.stream`으로 에이전트를 호출하는 것에 주목하세요.
    - 이 명령과 `pretty_print`는 모델과 도구 간에 정보를 전달하는 **메시지**를 표시합니다.
- 에이전트가 호출 간에 스키마를 기억하지 못한다는 점에 주목하세요... 이에 대해서는 나중에 더 다룹니다!

In [ ]:
question = "Which table has the largest number of entries?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "Which genre on average has the longest tracks?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "Please list all of the tables"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

**Create your own query here!**  Add some questions of your own.

**여기서 직접 쿼리를 만들어보세요!** 자신만의 질문을 추가해보세요.

do you know about a customer frank harris?

tell me about frank harris recent purchase history.

In [ ]:
question = "TRY YOUR OWN QUERY HERE"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()